# Scrape mountainproject.com
This notebook extracts data about climbing routes on mountainproject.com and climbers who have rated those routes. The steps are as follows:
1. Use Mountain Project's [route finder](https://www.mountainproject.com/route-guide) to extract all climb's in: New York -> Gunks, The -> Trapps, The. The url's of these routes are given [here](https://www.mountainproject.com/route-finder?selectedIds=105798818&type=rock&diffMinrock=800&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12400&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating).
2. Go to the page of each route and extract all useful data.
3. Go to the ratings page for each route and get each climbers' rating of the route.
4. Get all unique climbers that have rated the routes and extract data from their pages.

In [1]:
%pylab inline

# Plotting
import matplotlib.pyplot as plt

# Download html of webpage
import urllib2
# Beautiful soup
import bs4 as bs

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# 1. Extract route urls

In [3]:
def extract_route_urls(route_table_url, outfile):
    """Take the url for the mountainproject.com table of routes,
    and extract the url for each route.
    """
    source = urllib2.urlopen(route_table_url).read()
    soup = bs.BeautifulSoup(source, 'lxml')

    # All the route urls are repeated twice. Look at the ones in the table only.
    tables = soup.find_all('table')
    route_table = list(tables)[0]
    trs = route_table.find_all('tr')
    routes = [tr.find_all('td') for tr in trs]
    
    nroutes = len(routes)
    
    route_urls = []
    for i in range(nroutes):
        route = routes[i][0]
        s = str(route)
        url = s.split('href="')[1].split('">')[0]
        route_urls.append(url)
        
    df_routes = pd.DataFrame(data=route_urls, columns=['route_url'])
    df_routes.to_csv(outfile, index=False)

In [4]:
route_table_url = 'https://www.mountainproject.com/route-finder?diffMaxaid=75260&diffMaxboulder=21400&diffMaxice=38500&diffMaxmixed=60000&diffMaxrock=12400&diffMinaid=70000&diffMinboulder=20000&diffMinice=30000&diffMinmixed=50000&diffMinrock=800&is_sport_climb=1&is_top_rope=1&is_trad_climb=1&pitches=0&selectedIds=105798818&sort1=area&sort2=rating&stars=0&type=rock&viewAll=1'
outfile = '../data/route_urls.csv'
extract_route_urls(route_table_url, outfile)

In [9]:
df_route_urls = pd.read_csv(outfile)
df_route_urls.head(3)

,route_url
0,https://www.mountainproject.com/route/108300801/sudoriferous
1,https://www.mountainproject.com/route/112109817/cordelette-arete
2,https://www.mountainproject.com/route/106133868/69


# 2. Extract route data

In [6]:
def extract_route_data(route_url):
    """Extract interesting data from the webpage of a route.
    This is used for building the content based recommender.
    """
    source = urllib2.urlopen(route_url).read()
    soup = bs.BeautifulSoup(source, 'lxml')
    
    # Name of the route
    name_words = soup.find('div', class_='col-md-9 float-md-right').find('h1').text.split()
    route_name = ' '.join(name_words)
    
    # YDS difficulty rating
    difficulty = soup.find('span', class_='rateYDS').text.split()[0]
    
    # Text description of route
    description = soup.find('div', class_='fr-view').text
    
    # protection type, route length, number of pitches
    details = soup.find('table', class_='description-details')
    l = details.find('tr').text.split()
    
    pro_type = str(l[1].replace(',', ''))
    
    # If number of pitches is not stated, there is only 1
    indices = [i for i, s in enumerate(l) if 'pitch'.lower() in s]
    if indices == []:
        pitches = 1
    else:
        pitches = l[indices[0]-1]
    
    # If the word 'ft' does not appear, the length is unknown. Set it to 0 for now.
    indices = [i for i, s in enumerate(l) if 'ft'.lower() in s]
    if indices == []:
        length = 0
    else:
        length = l[indices[0]-1]
    
    
    # Total number of webpage views, monthly views
    l = details.find_all('tr')[2].text.split()
    total_views = int(l[2].replace(',', ''))
    monthly_views = int(l[5].split('/')[0].replace(',', ''))
    
    # Number of route ratings
    l = soup.find('a', class_='show-tooltip').text.split()
    nratings = int(l[-2].replace(',', ''))
    
    return {
        'route_name':route_name, 'difficulty':difficulty, 'description':description, 
        'pro_type':pro_type, 'length':length, 'pitches':pitches,
        'total_views':total_views, 'monthly_views':monthly_views, 
        'nratings':nratings}

In [7]:
def extract_all_routes_data(route_url_file, route_data_file):
    """Take route urls from a the csv file routes_url_file.
    Extract all the data for each route.
    Save it in the csv file routes_data_file.
    """
    df_routes = pd.read_csv(route_url_file)
    route_urls = df_routes['route_url'].values
    
    route_data_list = []
    nroutes = len(route_urls)
    for i in range(nroutes):
        url = route_urls[i]
        print(i, url)
        data = extract_route_data(url)
        data['iid'] = i
        data['route_url'] = url
        route_data_list.append(data)
    
    df = pd.DataFrame(route_data_list)
    df.to_csv(route_data_file, index=False, encoding='utf-8')

In [8]:
route_url_file = '../data/route_urls.csv'
route_data_file = '../data/route_data.csv'
extract_all_routes_data(route_url_file, route_data_file)

(0, 'https://www.mountainproject.com/route/108300801/sudoriferous')
(1, 'https://www.mountainproject.com/route/112109817/cordelette-arete')
(2, 'https://www.mountainproject.com/route/106133868/69')
(3, 'https://www.mountainproject.com/route/106437310/easy-keyhole')
(4, 'https://www.mountainproject.com/route/109528000/short-and-simple')
(5, 'https://www.mountainproject.com/route/107816567/short-job')
(6, 'https://www.mountainproject.com/route/105799721/horseman')
(7, 'https://www.mountainproject.com/route/106133879/no-picnic')
(8, 'https://www.mountainproject.com/route/106803063/black-fly')
(9, 'https://www.mountainproject.com/route/106133896/double-chin')
(10, 'https://www.mountainproject.com/route/105888225/bunny')
(11, 'https://www.mountainproject.com/route/110494047/pony-express')
(12, 'https://www.mountainproject.com/route/106133888/fancy-idiot')
(13, 'https://www.mountainproject.com/route/106138056/eyebrow')
(14, 'https://www.mountainproject.com/route/106281540/katzenjammer')
(15,

(120, 'https://www.mountainproject.com/route/105811994/ribs')
(121, 'https://www.mountainproject.com/route/107852049/simple-ceilings')
(122, 'https://www.mountainproject.com/route/105954124/high-corner')
(123, 'https://www.mountainproject.com/route/105799687/shockleys-ceiling')
(124, 'https://www.mountainproject.com/route/105902104/ribless-aka-spare-ribs')
(125, 'https://www.mountainproject.com/route/105799704/strictly-from-nowhere')
(126, 'https://www.mountainproject.com/route/107098017/gorilla-my-dreams')
(127, 'https://www.mountainproject.com/route/106134457/calisthenic')
(128, 'https://www.mountainproject.com/route/106295786/oscar-and-charlie')
(129, 'https://www.mountainproject.com/route/109022777/gaston')
(130, 'https://www.mountainproject.com/route/106134470/travels-with-charley')
(131, 'https://www.mountainproject.com/route/106031516/anguish')
(132, 'https://www.mountainproject.com/route/109555637/glyptodon')
(133, 'https://www.mountainproject.com/route/106295806/glypnod')
(134

(235, 'https://www.mountainproject.com/route/106565362/beyond-the-fringe')
(236, 'https://www.mountainproject.com/route/106146808/gory-thumb')
(237, 'https://www.mountainproject.com/route/106146816/badfinger')
(238, 'https://www.mountainproject.com/route/108242249/blunderbus')
(239, 'https://www.mountainproject.com/route/105920916/commando-rave')
(240, 'https://www.mountainproject.com/route/112217799/galactic-hitchhikers')
(241, 'https://www.mountainproject.com/route/106146799/wild-horses')
(242, 'https://www.mountainproject.com/route/105799095/never-never-land')
(243, 'https://www.mountainproject.com/route/107466579/nevermore')
(244, 'https://www.mountainproject.com/route/106189152/city-streets')
(245, 'https://www.mountainproject.com/route/105920912/balrog')
(246, 'https://www.mountainproject.com/route/106291093/dis-mantel')
(247, 'https://www.mountainproject.com/route/107226291/dat-mantel')
(248, 'https://www.mountainproject.com/route/106046438/jaccuse')
(249, 'https://www.mountainp

(351, 'https://www.mountainproject.com/route/106018746/the-spring-p1')
(352, 'https://www.mountainproject.com/route/115325123/mona')
(353, 'https://www.mountainproject.com/route/106733159/high-jinx')
(354, 'https://www.mountainproject.com/route/106089957/lito-and-the-swan')
(355, 'https://www.mountainproject.com/route/106813976/lost-in-yellow')
(356, 'https://www.mountainproject.com/route/113603406/tweak-or-freak')
(357, 'https://www.mountainproject.com/route/107516922/vader')
(358, 'https://www.mountainproject.com/route/108813461/manly-yes-but-i-like-it-too')
(359, 'https://www.mountainproject.com/route/106238729/the-winter')
(360, 'https://www.mountainproject.com/route/105798933/carbs-and-caffeine')
(361, 'https://www.mountainproject.com/route/105888230/the-fall')
(362, 'https://www.mountainproject.com/route/105798938/no-mans-land')
(363, 'https://www.mountainproject.com/route/105798924/the-yellow-wall')
(364, 'https://www.mountainproject.com/route/105898985/the-sting')
(365, 'https:

(466, 'https://www.mountainproject.com/route/109055246/shitty-mitty')
(467, 'https://www.mountainproject.com/route/106640673/hudsons-boulder-problem-aka-trashcan-overhang')
(468, 'https://www.mountainproject.com/route/106456276/low-exposure')
(469, 'https://www.mountainproject.com/route/112930386/freebie')


In [16]:
df_routes = pd.read_csv(route_data_file)

df_routes.head(2)

,description,difficulty,iid,length,monthly_views,nratings,pitches,pro_type,route_name,route_url,total_views
0,Pitch 1: Climb the face just next to the huge left-facing corner (feel free to use the corner bl...,5.2,0,120,19,7,2,Trad,Sudoriferous,https://www.mountainproject.com/route/108300801/sudoriferous,1309
1,"The first* route in the Trapps! An easy jaunt up a sharp arete, and surprisingly airy for a such...",5.2,1,80,38,12,1,Trad,Cordelette Arete,https://www.mountainproject.com/route/112109817/cordelette-arete,1161


# 3. Extract the ratings for each route

In [17]:
def extract_route_ratings(route_url):
    """Get all available users' ratings for the route.
    """
    # Website with the rating stats corresponding to route_url
    pieces = route_url.split('/')
    url = 'https://www.mountainproject.com/route/stats/{}/{}'.format(
        pieces[-2], pieces[-1])
    
    source = urllib2.urlopen(url).read()
    soup = bs.BeautifulSoup(source, 'lxml')
    
    # Each row of this table contains the user and number of stars
    rows = soup.find('table', class_='table table-striped').find_all('tr')
    nrows = len(rows)
    
    ratings = []
    for i in range(nrows):
        # Table row
        tr = rows[i]
        user_url = tr.find('a').get('href')
        # Each star is an image. Count the number of them.
        # A bomb image is the same as counting 0 stars.
        images = tr.find_all('img')
        images = [i.get('src') for i in images]
        star_img = 'https://cdn.apstatic.com/img/stars/starBlue.svg'
        nstars = images.count(star_img)
        d = {'user_url':user_url, 'route_url':route_url, 'rating':nstars}
        ratings.append(d)
        
    return ratings

In [18]:
def extract_ratings_for_all_routes(route_url_file, ratings_file):
    """Take route urls from a the csv file route_url_file.
    Extract the ratings for all of the routes.
    Save it in the csv file ratings_file.
    """
    df_routes = pd.read_csv(route_url_file)
    route_urls = df_routes['route_url'].values
    
    ratings_all_routes = []
    nroutes = len(route_urls)
    for i in range(nroutes):
        url = route_urls[i]
        print(i, url)
        ratings = extract_route_ratings(url)
        # Concatenate lists. Don't make a list of lists by appending.
        ratings_all_routes += ratings
        
    df = pd.DataFrame(ratings_all_routes)
    df.to_csv(ratings_file, index=False)

In [19]:
route_url_file = '../data/route_urls.csv'
ratings_file = '../data/route_ratings.csv'
extract_ratings_for_all_routes(route_url_file, ratings_file)

(0, 'https://www.mountainproject.com/route/108300801/sudoriferous')
(1, 'https://www.mountainproject.com/route/112109817/cordelette-arete')
(2, 'https://www.mountainproject.com/route/106133868/69')
(3, 'https://www.mountainproject.com/route/106437310/easy-keyhole')
(4, 'https://www.mountainproject.com/route/109528000/short-and-simple')
(5, 'https://www.mountainproject.com/route/107816567/short-job')
(6, 'https://www.mountainproject.com/route/105799721/horseman')
(7, 'https://www.mountainproject.com/route/106133879/no-picnic')
(8, 'https://www.mountainproject.com/route/106803063/black-fly')
(9, 'https://www.mountainproject.com/route/106133896/double-chin')
(10, 'https://www.mountainproject.com/route/105888225/bunny')
(11, 'https://www.mountainproject.com/route/110494047/pony-express')
(12, 'https://www.mountainproject.com/route/106133888/fancy-idiot')
(13, 'https://www.mountainproject.com/route/106138056/eyebrow')
(14, 'https://www.mountainproject.com/route/106281540/katzenjammer')
(15,

(120, 'https://www.mountainproject.com/route/105811994/ribs')
(121, 'https://www.mountainproject.com/route/107852049/simple-ceilings')
(122, 'https://www.mountainproject.com/route/105954124/high-corner')
(123, 'https://www.mountainproject.com/route/105799687/shockleys-ceiling')
(124, 'https://www.mountainproject.com/route/105902104/ribless-aka-spare-ribs')
(125, 'https://www.mountainproject.com/route/105799704/strictly-from-nowhere')
(126, 'https://www.mountainproject.com/route/107098017/gorilla-my-dreams')
(127, 'https://www.mountainproject.com/route/106134457/calisthenic')
(128, 'https://www.mountainproject.com/route/106295786/oscar-and-charlie')
(129, 'https://www.mountainproject.com/route/109022777/gaston')
(130, 'https://www.mountainproject.com/route/106134470/travels-with-charley')
(131, 'https://www.mountainproject.com/route/106031516/anguish')
(132, 'https://www.mountainproject.com/route/109555637/glyptodon')
(133, 'https://www.mountainproject.com/route/106295806/glypnod')
(134

(235, 'https://www.mountainproject.com/route/106565362/beyond-the-fringe')
(236, 'https://www.mountainproject.com/route/106146808/gory-thumb')
(237, 'https://www.mountainproject.com/route/106146816/badfinger')
(238, 'https://www.mountainproject.com/route/108242249/blunderbus')
(239, 'https://www.mountainproject.com/route/105920916/commando-rave')
(240, 'https://www.mountainproject.com/route/112217799/galactic-hitchhikers')
(241, 'https://www.mountainproject.com/route/106146799/wild-horses')
(242, 'https://www.mountainproject.com/route/105799095/never-never-land')
(243, 'https://www.mountainproject.com/route/107466579/nevermore')
(244, 'https://www.mountainproject.com/route/106189152/city-streets')
(245, 'https://www.mountainproject.com/route/105920912/balrog')
(246, 'https://www.mountainproject.com/route/106291093/dis-mantel')
(247, 'https://www.mountainproject.com/route/107226291/dat-mantel')
(248, 'https://www.mountainproject.com/route/106046438/jaccuse')
(249, 'https://www.mountainp

(351, 'https://www.mountainproject.com/route/106018746/the-spring-p1')
(352, 'https://www.mountainproject.com/route/115325123/mona')
(353, 'https://www.mountainproject.com/route/106733159/high-jinx')
(354, 'https://www.mountainproject.com/route/106089957/lito-and-the-swan')
(355, 'https://www.mountainproject.com/route/106813976/lost-in-yellow')
(356, 'https://www.mountainproject.com/route/113603406/tweak-or-freak')
(357, 'https://www.mountainproject.com/route/107516922/vader')
(358, 'https://www.mountainproject.com/route/108813461/manly-yes-but-i-like-it-too')
(359, 'https://www.mountainproject.com/route/106238729/the-winter')
(360, 'https://www.mountainproject.com/route/105798933/carbs-and-caffeine')
(361, 'https://www.mountainproject.com/route/105888230/the-fall')
(362, 'https://www.mountainproject.com/route/105798938/no-mans-land')
(363, 'https://www.mountainproject.com/route/105798924/the-yellow-wall')
(364, 'https://www.mountainproject.com/route/105898985/the-sting')
(365, 'https:

(466, 'https://www.mountainproject.com/route/109055246/shitty-mitty')
(467, 'https://www.mountainproject.com/route/106640673/hudsons-boulder-problem-aka-trashcan-overhang')
(468, 'https://www.mountainproject.com/route/106456276/low-exposure')
(469, 'https://www.mountainproject.com/route/112930386/freebie')


In [25]:
df_ratings = pd.read_csv(ratings_file)
# df_ratings.style.format(make_clickable, subset=['user_url', 'route_url'])
df_ratings.sample(3)

,rating,route_url,user_url
5971,4,https://www.mountainproject.com/route/105803228/stirrup-trouble,https://www.mountainproject.com/user/107238223/tony-lopez
2948,4,https://www.mountainproject.com/route/105799032/nosedive,https://www.mountainproject.com/user/107026497/jason-shermer
6265,2,https://www.mountainproject.com/route/106034654/easy-overhang,https://www.mountainproject.com/user/106468686/tdavidock


# 4. Extract data about each unique climber

In [26]:
def extract_user_data(user_url):
    """Get info about each user.
    """
    source = urllib2.urlopen(user_url).read()
    soup = bs.BeautifulSoup(source, 'lxml')
    
    # Get user name
    name = soup.find('h2', class_='dont-shrink mb-0').text
    
    # All info under the name
    profile_list = soup.find('div', class_='col-xs-12 text-xs-center').find('div').text.split()
    profile = ' '.join(profile_list)

    return {'name':name, 'profile':profile}

In [27]:
def extract_all_users_data(route_ratings_file, user_data_file):
    """Take unique user urls from a the csv file route_ratings_file.
    Extract all the data for each user.
    Save it in the csv file user_data_file.
    """
    df_ratings = pd.read_csv(route_ratings_file)
    
    # Get only the unique users
    user_urls = df_ratings['user_url'].unique()
    print user_urls.shape
    user_data_list = []
    nusers = len(user_urls)
    for i in range(nusers):
        url = user_urls[i]
        print(i, url)
        data = extract_user_data(url)
        data['uid'] = i
        data['user_url'] = url
        user_data_list.append(data)
    
    df = pd.DataFrame(user_data_list)
    df.to_csv(user_data_file, index=False, encoding='utf-8')

In [28]:
route_ratings_file = '../data/route_ratings.csv'
user_data_file = '../data/user_data.csv'
extract_all_users_data(route_ratings_file, user_data_file)

(2432,)
(0, 'https://www.mountainproject.com/user/106087077/mark-roth')
(1, 'https://www.mountainproject.com/user/109581184/joel-ryan')
(2, 'https://www.mountainproject.com/user/200223567/anyo-lesiuk')
(3, 'https://www.mountainproject.com/user/107253738/dragons')
(4, 'https://www.mountainproject.com/user/106630260/j-serpico')
(5, 'https://www.mountainproject.com/user/110409351/ben-hoste')
(6, 'https://www.mountainproject.com/user/109206590/john271')
(7, 'https://www.mountainproject.com/user/112449870/david-kerkeslager')
(8, 'https://www.mountainproject.com/user/112460885/nicole-vidal')
(9, 'https://www.mountainproject.com/user/113522523/shionanlim')
(10, 'https://www.mountainproject.com/user/106679659/jonathan-steitzer')
(11, 'https://www.mountainproject.com/user/107518458/kurtz')
(12, 'https://www.mountainproject.com/user/112423730/albi-eds')
(13, 'https://www.mountainproject.com/user/106646308/kiff')
(14, 'https://www.mountainproject.com/user/110729900/j-l')
(15, 'https://www.mountai

(123, 'https://www.mountainproject.com/user/106567319/mike-mclean')
(124, 'https://www.mountainproject.com/user/106762873/eric-albers')
(125, 'https://www.mountainproject.com/user/13658/rob-bauer')
(126, 'https://www.mountainproject.com/user/106607660/johnwesely')
(127, 'https://www.mountainproject.com/user/106822110/kevin-heckeler')
(128, 'https://www.mountainproject.com/user/106797746/valerie-ab')
(129, 'https://www.mountainproject.com/user/106863637/adkeditor-brown')
(130, 'https://www.mountainproject.com/user/106456515/zach-s')
(131, 'https://www.mountainproject.com/user/107872722/stephen-bittner')
(132, 'https://www.mountainproject.com/user/108674850/ccas')
(133, 'https://www.mountainproject.com/user/109405478/patrick-killian')
(134, 'https://www.mountainproject.com/user/106993942/luc-514')
(135, 'https://www.mountainproject.com/user/108259746/jim-thayer')
(136, 'https://www.mountainproject.com/user/105890911/greg-davis')
(137, 'https://www.mountainproject.com/user/110036457/griff

(245, 'https://www.mountainproject.com/user/10610/jay-knower')
(246, 'https://www.mountainproject.com/user/106011891/ross-purnell')
(247, 'https://www.mountainproject.com/user/106687445/peterw-whitmore')
(248, 'https://www.mountainproject.com/user/106749092/brian-rosenfeld')
(249, 'https://www.mountainproject.com/user/106619581/greg-sudlow')
(250, 'https://www.mountainproject.com/user/106751248/rcongo')
(251, 'https://www.mountainproject.com/user/106588732/lee-h')
(252, 'https://www.mountainproject.com/user/106715896/christina-callaghan')
(253, 'https://www.mountainproject.com/user/106773591/jonathan-ward')
(254, 'https://www.mountainproject.com/user/11156/tradiban')
(255, 'https://www.mountainproject.com/user/106802916/anton-tokranov')
(256, 'https://www.mountainproject.com/user/106113062/ben-c')
(257, 'https://www.mountainproject.com/user/11690/rui-ferreira')
(258, 'https://www.mountainproject.com/user/106680875/david-ford')
(259, 'https://www.mountainproject.com/user/106844475/nate-

(367, 'https://www.mountainproject.com/user/108321835/sam-fox')
(368, 'https://www.mountainproject.com/user/111126783/city-dweller')
(369, 'https://www.mountainproject.com/user/110538349/liz-reiman')
(370, 'https://www.mountainproject.com/user/110148121/systematic')
(371, 'https://www.mountainproject.com/user/109688349/ajg519')
(372, 'https://www.mountainproject.com/user/108052600/liz-han')
(373, 'https://www.mountainproject.com/user/108085357/tyler-kempney')
(374, 'https://www.mountainproject.com/user/109620395/maureen-petterson')
(375, 'https://www.mountainproject.com/user/109711410/billy-simek')
(376, 'https://www.mountainproject.com/user/107755397/ed-wade')
(377, 'https://www.mountainproject.com/user/109411206/cameron-collins')
(378, 'https://www.mountainproject.com/user/109634018/richard-beck')
(379, 'https://www.mountainproject.com/user/111626699/alessandro-pruscino')
(380, 'https://www.mountainproject.com/user/111627067/jilliancai')
(381, 'https://www.mountainproject.com/user/11

(488, 'https://www.mountainproject.com/user/109542989/vadim-mikheev')
(489, 'https://www.mountainproject.com/user/200409293/vinny-casey')
(490, 'https://www.mountainproject.com/user/200067404/max-zielinski')
(491, 'https://www.mountainproject.com/user/200062780/aare-puussaar')
(492, 'https://www.mountainproject.com/user/107551540/zyvandus')
(493, 'https://www.mountainproject.com/user/107552617/b-callahan')
(494, 'https://www.mountainproject.com/user/10909/charlesr')
(495, 'https://www.mountainproject.com/user/106125724/ian-wauchope')
(496, 'https://www.mountainproject.com/user/107519230/adam-faller')
(497, 'https://www.mountainproject.com/user/106030894/brian-aitken')
(498, 'https://www.mountainproject.com/user/107607088/danielle-nelson')
(499, 'https://www.mountainproject.com/user/107664805/nicholas-kinloch')
(500, 'https://www.mountainproject.com/user/105832216/buckeeb-hart')
(501, 'https://www.mountainproject.com/user/105922412/nick-weinstock')
(502, 'https://www.mountainproject.com

...

(1932, 'https://www.mountainproject.com/user/106013776/charlie-s')
(1933, 'https://www.mountainproject.com/user/107567801/steve-graham')
(1934, 'https://www.mountainproject.com/user/105924373/mbx5-barnas')
(1935, 'https://www.mountainproject.com/user/111820491/john-stoneham')
(1936, 'https://www.mountainproject.com/user/106568741/michal-pasniewski')
(1937, 'https://www.mountainproject.com/user/107082768/johnva')
(1938, 'https://www.mountainproject.com/user/107865932/nicole-e-murray')
(1939, 'https://www.mountainproject.com/user/110590584/mikey-anderson')
(1940, 'https://www.mountainproject.com/user/110781286/chris-moratz')
(1941, 'https://www.mountainproject.com/user/112009752/jonathan-parham')
(1942, 'https://www.mountainproject.com/user/109167471/stephen-montgomery')
(1943, 'https://www.mountainproject.com/user/105795871/tombo')
(1944, 'https://www.mountainproject.com/user/109199664/emily-black')
(1945, 'https://www.mountainproject.com/user/108901800/gabe-torres')
(1946, 'https://www

(2051, 'https://www.mountainproject.com/user/107536111/gary-jones')
(2052, 'https://www.mountainproject.com/user/111536914/allison-tau')
(2053, 'https://www.mountainproject.com/user/15142/adam-peters')
(2054, 'https://www.mountainproject.com/user/106919775/gwb')
(2055, 'https://www.mountainproject.com/user/107388687/jindap')
(2056, 'https://www.mountainproject.com/user/106315900/nate-miller')
(2057, 'https://www.mountainproject.com/user/112524899/matthew-stark')
(2058, 'https://www.mountainproject.com/user/200079884/caitlin-m')
(2059, 'https://www.mountainproject.com/user/111179858/wendyah')
(2060, 'https://www.mountainproject.com/user/200057419/alejandro-t')
(2061, 'https://www.mountainproject.com/user/108647959/izzy-nawfal')
(2062, 'https://www.mountainproject.com/user/112084693/andrew-s')
(2063, 'https://www.mountainproject.com/user/105937206/dan-ward')
(2064, 'https://www.mountainproject.com/user/107608024/joshua-corbin')
(2065, 'https://www.mountainproject.com/user/15148/jim-thomp

(2170, 'https://www.mountainproject.com/user/105822848/aeon-aki')
(2171, 'https://www.mountainproject.com/user/110501405/paul-northup')
(2172, 'https://www.mountainproject.com/user/108732453/wallace')
(2173, 'https://www.mountainproject.com/user/110739544/katie-moenkhaus')
(2174, 'https://www.mountainproject.com/user/107370769/josh-schmaltz')
(2175, 'https://www.mountainproject.com/user/105829653/tim-wolfe')
(2176, 'https://www.mountainproject.com/user/111282455/leann-du')
(2177, 'https://www.mountainproject.com/user/108436732/jeannier3')
(2178, 'https://www.mountainproject.com/user/11812/eli-helmuth')
(2179, 'https://www.mountainproject.com/user/106987324/jayz')
(2180, 'https://www.mountainproject.com/user/108358038/michael-burnell')
(2181, 'https://www.mountainproject.com/user/106257649/jason-killgore')
(2182, 'https://www.mountainproject.com/user/107133892/tomw')
(2183, 'https://www.mountainproject.com/user/111649446/nana-aoki')
(2184, 'https://www.mountainproject.com/user/105975214

(2289, 'https://www.mountainproject.com/user/200399257/marcello-cricco-lizza')
(2290, 'https://www.mountainproject.com/user/112496934/jesse-vanek')
(2291, 'https://www.mountainproject.com/user/106376808/jill-north-wilson')
(2292, 'https://www.mountainproject.com/user/112026181/kristinicole')
(2293, 'https://www.mountainproject.com/user/106000626/brianup-up')
(2294, 'https://www.mountainproject.com/user/105830665/ralph-kolva')
(2295, 'https://www.mountainproject.com/user/10359/leo-paik')
(2296, 'https://www.mountainproject.com/user/106714105/stevendo')
(2297, 'https://www.mountainproject.com/user/106068274/ryan-hunt')
(2298, 'https://www.mountainproject.com/user/107284917/tessa-o')
(2299, 'https://www.mountainproject.com/user/108184331/alexis-wojtanowski')
(2300, 'https://www.mountainproject.com/user/109081636/eberliner')
(2301, 'https://www.mountainproject.com/user/106043351/hans')
(2302, 'https://www.mountainproject.com/user/110518966/sonya-st-john')
(2303, 'https://www.mountainprojec

(2409, 'https://www.mountainproject.com/user/109740864/ryanmullins')
(2410, 'https://www.mountainproject.com/user/106319582/treker')
(2411, 'https://www.mountainproject.com/user/106935843/will-adsit')
(2412, 'https://www.mountainproject.com/user/108132717/aharlec')
(2413, 'https://www.mountainproject.com/user/108373969/hannah-tuttle')
(2414, 'https://www.mountainproject.com/user/112843735/chris-kocher')
(2415, 'https://www.mountainproject.com/user/110687338/ian-mcmullen')
(2416, 'https://www.mountainproject.com/user/107595240/kellen-j')
(2417, 'https://www.mountainproject.com/user/110802936/arthur-torrey')
(2418, 'https://www.mountainproject.com/user/105870073/jordan-k')
(2419, 'https://www.mountainproject.com/user/12291/jamie-givens')
(2420, 'https://www.mountainproject.com/user/106298729/k-man')
(2421, 'https://www.mountainproject.com/user/107550300/david-raines')
(2422, 'https://www.mountainproject.com/user/108156601/steven-scr')
(2423, 'https://www.mountainproject.com/user/10886726

# Add uid and iid to route_ratings.csv

In [31]:
route_data_file = '../data/route_data.csv'
df_routes = pd.read_csv(route_data_file)

ratings_file = '../data/route_ratings.csv'
df_ratings = pd.read_csv(ratings_file)

user_data_file = '../data/user_data.csv'
df_users = pd.read_csv(user_data_file)

# Start with the ratings table.
# Add the iid
df = df_ratings.merge(df_routes[['iid', 'route_url']], on='route_url', how='inner')
# Add the uid
df = df.merge(df_users[['uid', 'user_url']], on='user_url', how='inner')
# Sort for readability
df = df.sort_values(['iid', 'uid'])

# Save the new ratings file
df[['iid', 'uid', 'rating', 'route_url', 'user_url']].to_csv('../data/route_ratings_merged.csv', index=False, encoding='utf-8')

In [32]:
display(df_ratings.head())
display(df.head())

,rating,route_url,user_url
0,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/106087077/mark-roth
1,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/109581184/joel-ryan
2,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/200223567/anyo-lesiuk
3,1,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/107253738/dragons
4,1,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/106630260/j-serpico


,rating,route_url,user_url,iid,uid
0,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/106087077/mark-roth,0,0
142,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/109581184/joel-ryan,0,1
218,2,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/200223567/anyo-lesiuk,0,2
224,1,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/107253738/dragons,0,3
248,1,https://www.mountainproject.com/route/108300801/sudoriferous,https://www.mountainproject.com/user/106630260/j-serpico,0,4
